## StarTracker Coordinate Check - 08-Mar-23

Craig Lage - Double checking coordinate transformations

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.latiss import AUXTEL_LOCATION
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION

In [ ]:
dates = ['2023-03-09', '2023-03-15', '2023-03-21']
azs = []
alts = []
deltaAzs = []
deltaAlts = []

for date in dates:
    filename = f'/scratch/cslage/starTracker/startracker_{date}.json'
    frame = ICRS
    df = pd.read_json(filename)
    df = df.transpose()
    df = df.dropna()
    for seqNum in df.index.values.tolist():
        if date == '2023-03-09' and seqNum < 1138:
            continue
        row = df.loc[seqNum]
        skyLocation = SkyCoord(row['Ra']*u.deg, row['Dec']*u.deg, frame=frame)
        time = Time(row['MJD'], format='mjd', scale='tai')
        altAz1 = AltAz(obstime=time, location=SIMONYI_LOCATION)
        obsAltAz1 = skyLocation.transform_to(altAz1)
        print(f"Header AltAz values =             ({row['Alt']:.6f}, {row['Az']:.6f})")
        print(f"AltAz from header Ra/Dec values = ({obsAltAz1.alt.deg:.6f}, {obsAltAz1.az.deg:.6f})")
        deltaAlt = (row['Alt'] - obsAltAz1.alt.deg) * 3600.0
        deltaAz = (row['Az'] - obsAltAz1.az.deg) * 3600.0
        print(f"DeltaAlt = {deltaAlt:.1f} arcseconds, DeltaAz = {deltaAz:.1f} arcseconds")
        print()
        # Now go the other way
        altAz2 = SkyCoord(AltAz(alt=row['Alt']*u.deg, az=row['Az']*u.deg, obstime=time, location=SIMONYI_LOCATION))
        RaDec = altAz2.transform_to(frame)
        print(f"Header RA/Dec values =            ({row['Ra']:.6f}, {row['Dec']:.6f})")
        print(f"RA/Dec from header AltAz values = ({RaDec.ra.deg:.6f}, {RaDec.dec.deg:.6f})")
        deltaRA = (row['Ra'] - RaDec.ra.deg) * 3600.0
        deltaDec = (row['Dec'] - RaDec.dec.deg) * 3600.0
        print(f"DeltaRA = {deltaRA:.1f} arcseconds, DeltaDec = {deltaDec:.1f} arcseconds")
        print()
        azs.append(row['Az'] * np.pi / 180.0)
        alts.append(row['Alt'])
        deltaAzs.append(deltaAz)
        deltaAlts.append(deltaAlt)


In [ ]:
len(azs)

In [ ]:
fig=plt.figure(figsize=(16,16))
#plt.suptitle("RADec vs AltAz Error, 20230309")
#plt.subplots_adjust(wspace = 1.0)
vmin = -20.0
vmax = 20.0
levels = np.linspace(vmin, vmax, 51)
ax1 = plt.subplot(121, projection='polar')
ax1.set_title(f"Azimuth Error (arcseconds)\nMultiple dates")
ax1.invert_yaxis()
ax1.set_rlim(90,0)
contourf_1 = ax1.tricontourf(azs, alts, deltaAzs, levels=levels)
cax1 = fig.add_axes([ax1.get_position().x1+0.01,ax1.get_position().y0,0.02,ax1.get_position().height])
plt.colorbar(contourf_1, cax=cax1)
ax2 = plt.subplot(122, projection='polar')
ax2.set_title(f"Elevation Error (arcseconds)\nMultiple dates")
ax2.invert_yaxis()
ax2.set_rlim(90,0)
contourf_2 = ax2.tricontourf(azs, alts, deltaAlts, levels=levels)
cax2 = fig.add_axes([ax2.get_position().x1+0.01,ax2.get_position().y0,0.02,ax2.get_position().height])
plt.colorbar(contourf_2, cax=cax2)
plt.savefig(f'/scratch/cslage/starTracker/Header_Discrepancy_Multiple.png')

In [ ]:
np.max(deltaAlts)

In [ ]:
dates = ['2023-03-09', '2023-03-15', '2023-03-21']

for i, date in enumerate(dates):
    azs = []
    alts = []
    deltaAzs = []
    deltaAlts = []

    filename = f'/scratch/cslage/starTracker/startracker_{date}.json'
    frame = ICRS
    df = pd.read_json(filename)
    df = df.transpose()
    df = df.dropna()
    for seqNum in df.index.values.tolist():
        if date == '2023-03-09' and seqNum < 1138:
            continue
        row = df.loc[seqNum]
        skyLocation = SkyCoord(row['Ra']*u.deg, row['Dec']*u.deg, frame=frame)
        time = Time(row['MJD'], format='mjd', scale='tai')
        altAz1 = AltAz(obstime=time, location=SIMONYI_LOCATION)
        obsAltAz1 = skyLocation.transform_to(altAz1)
        deltaAlt = (row['Alt'] - obsAltAz1.alt.deg) * 3600.0
        deltaAz = (row['Az'] - obsAltAz1.az.deg) * 3600.0
        azs.append(row['Az'] * np.pi / 180.0)
        alts.append(row['Alt'])
        deltaAzs.append(deltaAz)
        deltaAlts.append(deltaAlt)
    plt.subplot(3,2,2*i+1)
    plt.title(f"{date}, deltaAzs")
    plt.hist(deltaAzs, bins=50, range=(-50,50))
    plt.xlim(-50,50)
    plt.xlabel("Header discrepancy(arcseconds)")
    plt.subplot(3,2,2*i+2)
    plt.title(f"{date}, deltaAlts")
    plt.hist(deltaAlts, bins=50, range=(-50,50))
    plt.xlim(-50,50)
    plt.xlabel("Header discrepancy(arcseconds)")
plt.subplots_adjust(hspace=1.0)
plt.savefig(f'/scratch/cslage/starTracker/Header_Discrepancy_Histograms.png')

In [ ]:
date = '2023-03-21'
i=0
azs = []
alts = []
deltaAzs = []
deltaAlts = []

filename = f'/scratch/cslage/starTracker/startracker_{date}.json'
frame = ICRS
df = pd.read_json(filename)
df = df.transpose()
df = df.dropna()
for seqNum in df.index.values.tolist():
    row = df.loc[seqNum]
    skyLocation = SkyCoord(row['Ra']*u.deg, row['Dec']*u.deg, frame=frame)
    time = Time(row['MJD'], format='mjd', scale='tai')
    altAz1 = AltAz(obstime=time, location=SIMONYI_LOCATION)
    obsAltAz1 = skyLocation.transform_to(altAz1)
    deltaAlt = (row['Alt'] - obsAltAz1.alt.deg) * 3600.0
    deltaAz = (row['Az'] - obsAltAz1.az.deg) * 3600.0
    azs.append(row['Az'] * np.pi / 180.0)
    alts.append(row['Alt'])
    deltaAzs.append(deltaAz)
    deltaAlts.append(deltaAlt)
plt.subplot(1,2,2*i+1)
plt.title(f"{date}, deltaAzs")
plt.hist(deltaAzs, bins=10, range=(-75,75))
plt.xlim(-100, 100)
plt.xlabel("Header discrepancy(arcseconds)")
plt.subplot(1,2,2*i+2)
plt.title(f"{date}, deltaAlts")
plt.hist(deltaAlts, bins=10, range=(-25,25))
plt.xlim(-25,25)
plt.xlabel("Header discrepancy(arcseconds)")
plt.subplots_adjust(hspace=1.0)
plt.savefig(f'/scratch/cslage/starTracker/Header_Discrepancy_Histograms_20230321.png')